In [8]:
import pandas as pd
import numpy as np

In [9]:
from gp_pref_elicit_luisa import gaussian_process as GP 
from gp_pref_elicit_luisa import dataset as data 
from gp_pref_elicit_luisa import acquisition_function as acquisition_function
from gp_pref_elicit_luisa.gp_utilities import utils_ccs as gp_utils_ccs
from gp_pref_elicit_luisa.gp_utilities import utils_data as gp_utils_data
from gp_pref_elicit_luisa.gp_utilities import utils_user as gp_utils_users

In [10]:
# gp_utils_ccs.get_ccs(2, 20)
# outputs a synthetic Pareto Coverage Set of value vectors with 2 objectives and 20 datapoints

The original dataset, in the data_preprocessed.ipynb file, can not be used for the Gaussian Process as it is not in the form of value vectors. To bring it into the form of value vectors, the dataset needs to be passed into a Multi-Objective Shortest Path Planning Solver which will output value vectors. These value vectors can then be an input for the GP. Thus, for now, I am using a synthetic Pareto Coverage Set (PCS) with 2 objectives and 20 datapoints which gives us synthetic value vectors for input to the GP.

In [11]:
synthetic_pcs = np.array([[0.14370116, 0.99159928],
       [0.9797389 , 0.2242916 ],
       [0.        , 1.        ],
       [0.91055917, 0.45020785],
       [0.59678925, 0.81854996],
       [1.        , 0.        ],
       [0.94198057, 0.352479  ],
       [0.81501748, 0.65358114],
       [0.99814566, 0.05429028],
       [0.33305315, 0.94955291],
       [0.28860669, 0.96123215],
       [0.99999796, 0.02591092],
       [0.98910769, 0.14092867],
       [0.18584726, 0.98334638],
       [0.05210043, 0.99838732],
       [0.87761802, 0.52114756],
       [0.74002719, 0.7144284 ],
       [0.21487083, 0.97724899],
       [0.43622937, 0.90230767],
       [0.99525346, 0.08213535]])
synthetic_pcs 

array([[0.14370116, 0.99159928],
       [0.9797389 , 0.2242916 ],
       [0.        , 1.        ],
       [0.91055917, 0.45020785],
       [0.59678925, 0.81854996],
       [1.        , 0.        ],
       [0.94198057, 0.352479  ],
       [0.81501748, 0.65358114],
       [0.99814566, 0.05429028],
       [0.33305315, 0.94955291],
       [0.28860669, 0.96123215],
       [0.99999796, 0.02591092],
       [0.98910769, 0.14092867],
       [0.18584726, 0.98334638],
       [0.05210043, 0.99838732],
       [0.87761802, 0.52114756],
       [0.74002719, 0.7144284 ],
       [0.21487083, 0.97724899],
       [0.43622937, 0.90230767],
       [0.99525346, 0.08213535]])

In [12]:
# initializing GP, Dataset and Acquisition Function 
# num_objectives = 2
GP = GP.GPPairwise(2)
utils_comparisons = data.DatasetPairwise(2)
acquisition_function_DA = acquisition_function.DiscreteAcquirer(input_domain=synthetic_pcs, query_type='pairwise', seed=None, acquisition_type='expected improvement')
acquisition_function_EI = acquisition_function.get_expected_improvement(datapoints=synthetic_pcs, gaussian_process=GP, datapoints_hist=acquisition_function_DA.history, xi=0.01)

In [13]:
# getting the user preferences for generating a ground truth utility function
user_pref = gp_utils_users.UserPreference(2, std_noise=0.1)

In [14]:
# getting the datapoints at which we need to query the user first 
start_points = acquisition_function_DA.get_start_points(gaussian_process=GP)
start_points


(array([0.94198057, 0.352479  ]), array([0.59678925, 0.81854996]))

In [15]:
# generating utilities for the starting datapoints
ground_truth_utility_func = user_pref.get_preference(start_points, add_noise=True)
ground_truth_utility_func

array([0.41127682, 0.43836451])

In [16]:
# ground_truth_utility_func = user_pref.get_preference(synthetic_pcs, add_noise=True)
# ground_truth_utility_func

In [17]:
highest_utility_index = np.argmax(ground_truth_utility_func)
lowest_utility_index = np.argmin(ground_truth_utility_func)
print('Highest utility is at index ', highest_utility_index,  'for the user in the dataset', '\n', 
'Lowest utility is at index ', lowest_utility_index, 'for the user in the dataset')
# this outputs the index of the points in the synthetic dataset which have the highest and lowest utility

Highest utility is at index  1 for the user in the dataset 
 Lowest utility is at index  0 for the user in the dataset


In [18]:
# getting the user preference for the datapoint in the synthetic dataset having the highest utility
# ground_truth_utility_func = user_pref.get_preference(synthetic_pcs[highest_utility_index], add_noise=True)
# ground_truth_utility_func

In [19]:
# comparing datapoints with highest and lowest utility values generated from the ground truth utility function 
# and adding these to the GP
utils_comparisons.add_single_comparison(synthetic_pcs[highest_utility_index], synthetic_pcs[lowest_utility_index])
utils_comparisons_datapoints = utils_comparisons.datapoints
utils_comparisons_datapoints 

array([[0.9797389 , 0.2242916 ],
       [0.14370116, 0.99159928]])

In [20]:
# to get the point which will be retained by the comparison in the dataset
highest_point = np.argmax(utils_comparisons_datapoints)
highest_point

3

In [21]:
# updating the GP based on the comparisons added to the dataset
GP.update(utils_comparisons)

In [22]:
# gives the next points to query for the user 
next_points = acquisition_function_DA.get_next_point(gaussian_process=GP, dataset=utils_comparisons)
next_points
# outputs next set of value vectors for comparisons

array([0.81501748, 0.65358114])

In [23]:
# we now run the user preference on the next points to query
ground_truth_utility_func = user_pref.get_preference(x=next_points, add_noise=True)
ground_truth_utility_func

array([0.54893453])

In [24]:
# we get the index of the highes point which indicates highest utility for the datapoint
highest_utility_index1 = np.argmax(ground_truth_utility_func)
highest_utility_index1

0

In [25]:
# comparing datapoints with highest and lowest utility values generated from the ground truth utility function 
# and adding these to the GP
utils_comparisons.add_single_comparison(synthetic_pcs[highest_point], synthetic_pcs[highest_utility_index1])
utils_comparisons_datapoints = utils_comparisons.datapoints
utils_comparisons_datapoints 

array([[0.9797389 , 0.2242916 ],
       [0.14370116, 0.99159928],
       [0.91055917, 0.45020785]])

In [26]:
GP.update(utils_comparisons)

In [27]:
# points to exclude for getting the expected improvement
# we do this because we want to exclude all the points the current maximum was already compared to
exclude = acquisition_function.exclude_points_pairwise(dataset=utils_comparisons)
exclude

array([[0.91055917, 0.45020785],
       [0.14370116, 0.99159928]])

In [28]:
next_point_EI = acquisition_function_DA.get_next_point_EI(gaussian_process=GP, exclude=exclude)
next_point_EI

array([0.59678925, 0.81854996])

In [29]:
# generating utilities for the next datapoints according to EI
ground_truth_utility_func1 = user_pref.get_preference(next_point_EI, add_noise=True)
ground_truth_utility_func1

array([0.79790602])

In [30]:
# we get the index of the highes point which indicates highest utility for the datapoint
highest_utility_index2 = np.argmax(ground_truth_utility_func1)
highest_utility_index2

0

In [31]:
# comparing datapoints with highest and lowest utility values generated from the ground truth utility function 
# and adding these to the GP
utils_comparisons.add_single_comparison(synthetic_pcs[highest_point], synthetic_pcs[highest_utility_index1])
utils_comparisons_datapoints = utils_comparisons.datapoints
utils_comparisons_datapoints 

array([[0.9797389 , 0.2242916 ],
       [0.14370116, 0.99159928],
       [0.91055917, 0.45020785]])

In [32]:
GP.update(utils_comparisons)

In [33]:
# similarly for this, I am not sure if the input and output are correct
acquisition_function.get_expected_improvement(datapoints=synthetic_pcs, gaussian_process=GP, datapoints_hist=acquisition_function_DA.history, xi=0.01)

array([0.08134857, 0.12136644, 0.17213742, 0.12136643, 0.23883934,
       0.22413789, 0.17184728, 0.22396078, 0.19918195, 0.20525771,
       0.17534387, 0.21428428, 0.13734255, 0.09294928, 0.12794248,
       0.13376841, 0.23767833, 0.1123285 , 0.23470928, 0.18036713])

In [34]:
prob_impr = acquisition_function.get_probability_of_improvement(x=next_point_EI, gaussian_process=GP, x_previous=next_points)
prob_impr

array([0.49035336])

In [ ]:
# while prob_impr>0.05 :



In [ ]:
# I haven't updated this for loop according to the code above. Once I know the code above is correct, I will put everything here
num_iter = 2
for i in range(num_iter):
    # sampling on synthetic pcs according to gaussian process
    sampled_points_GP = GP.sample(synthetic_pcs)
    # output of this sampling is the utility values

    # Getting highest and lowest utility values from the sampled points
    highest_util = max(sampled_points_GP)
    lowest_util = min(sampled_points_GP)

    # performing single comparisons on the highest and lowest utility values
    
print('Sampled points from the GP are: \n', sampled_points_GP, '\n' 
    'Highest utility value is: ', highest_util, '\n'
    'Lowest utility value is: ', lowest_util, '\n')

Sampled points from the GP are: 
 [ 0.67982393  0.37691511 -0.54999997 -0.03992034  0.64456674  0.22169369
  0.3489296   0.10824162  0.17637865 -0.46793137 -0.18302655  0.19235659
  0.28728856  0.64440499  0.00787193 -0.52657715  0.45506158  0.4796792
  0.04150372  0.18796297] 
Highest utility value is:  0.6798239315859632 
Lowest utility value is:  -0.5499999659741701 

